<a href="https://colab.research.google.com/github/OWL-408/-1/blob/main/Excel%E2%86%92csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files, drive
import io
import os

# Google Driveをマウント
drive.mount('/content/drive')

# 保存先ディレクトリの指定
save_dir = '/content/drive/My Drive/Excel_to_CSV'

# 保存先ディレクトリが存在しない場合は作成
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# ローカルからExcelファイルをアップロード
uploaded = files.upload()

# アップロードされたファイルの処理
for filename, content in uploaded.items():
    # Excelファイルを読み込む
    try:
        df = pd.read_excel(io.BytesIO(content))
    except Exception as e:
        print(f"エラー: {filename} の読み込みに失敗しました。理由: {str(e)}")
        continue

    # ファイル名から拡張子を除去し、新しいCSVファイル名を作成
    csv_filename = os.path.splitext(filename)[0] + '.csv'

    # CSVファイルとして保存（UTF-8エンコーディング）
    csv_path = os.path.join(save_dir, csv_filename)
    try:
        df.to_csv(csv_path, index=False, encoding='utf-8')
        print(f'{filename} を {csv_filename} として {save_dir} に保存しました。')
    except Exception as e:
        print(f"エラー: {csv_filename} の保存に失敗しました。理由: {str(e)}")

print("処理が完了しました。")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving F-P.xlsx to F-P.xlsx
Saving F.xlsx to F.xlsx
Saving M.xlsx to M.xlsx
F-P.xlsx を F-P.csv として /content/drive/My Drive/Excel_to_CSV に保存しました。
F.xlsx を F.csv として /content/drive/My Drive/Excel_to_CSV に保存しました。
M.xlsx を M.csv として /content/drive/My Drive/Excel_to_CSV に保存しました。
処理が完了しました。


Ver2


In [ ]:
import pandas as pd
from google.colab import files, drive
import io
import os
import re

# Google Driveをマウント
drive.mount('/content/drive')

# 保存先ディレクトリの指定
save_dir = '/content/drive/My Drive/Excel_to_CSV'

# 保存先ディレクトリが存在しない場合は作成
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# ローカルからExcelファイルをアップロード
uploaded = files.upload()

# アップロードされたファイルの処理
for filename, content in uploaded.items():
    # ファイル名から番号を除去
    base_filename = re.sub(r' \(\d+\)', '', filename)

    # Excelファイルを読み込む
    try:
        df = pd.read_excel(io.BytesIO(content))
    except Exception as e:
        print(f"エラー: {base_filename} の読み込みに失敗しました。理由: {str(e)}")
        continue

    # time列とPeak Strength Time列の処理
    date_columns = ['time', 'Peak Strength Time']
    for col in date_columns:
        if col in df.columns:
            # データを datetime 型に変換し、ISO 8601 形式の文字列に変換
            df[col] = pd.to_datetime(df[col]).apply(lambda x: x.isoformat(timespec='milliseconds') if pd.notnull(x) else x)

    # D列（Nominal stress）を削除
    if 'Nominal stress' in df.columns:
        df = df.drop('Nominal stress', axis=1)

    # ファイル名から拡張子を除去し、新しいCSVファイル名を作成
    csv_filename = os.path.splitext(base_filename)[0] + '.csv'

    # CSVファイルとして保存（UTF-8エンコーディング）
    csv_path = os.path.join(save_dir, csv_filename)
    try:
        # 既存のファイルがある場合は上書き
        df.to_csv(csv_path, index=False, encoding='utf-8', mode='w')
        print(f'{base_filename} を {csv_filename} として {save_dir} に保存しました。')
    except Exception as e:
        print(f"エラー: {csv_filename} の保存に失敗しました。理由: {str(e)}")

print("処理が完了しました。")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving B.xlsx to B (7).xlsx
Saving F.xlsx to F (4).xlsx
Saving F-P.xlsx to F-P (4).xlsx
Saving M.xlsx to M (4).xlsx
B.xlsx を B.csv として /content/drive/My Drive/Excel_to_CSV に保存しました。
F.xlsx を F.csv として /content/drive/My Drive/Excel_to_CSV に保存しました。
F-P.xlsx を F-P.csv として /content/drive/My Drive/Excel_to_CSV に保存しました。
M.xlsx を M.csv として /content/drive/My Drive/Excel_to_CSV に保存しました。
処理が完了しました。


ver3_time列の形式の修正

In [2]:
import pandas as pd
from google.colab import files, drive
import io
import os
import re

# Google Driveをマウント
drive.mount('/content/drive')

# 保存先ディレクトリの指定
save_dir = '/content/drive/My Drive/Excel_to_CSV'

# 保存先ディレクトリが存在しない場合は作成
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# ローカルからExcelファイルをアップロード
uploaded = files.upload()

def clean_datetime(dt):
    try:
        return pd.to_datetime(dt, format='%Y-%m-%d %H:%M:%S.%f')
    except ValueError:
        try:
            return pd.to_datetime(dt, format='%Y-%m-%d %H:%M:%S')
        except ValueError:
            return pd.NaT

# アップロードされたファイルの処理
for filename, content in uploaded.items():
    # ファイル名から番号を除去
    base_filename = re.sub(r' \(\d+\)', '', filename)

    # Excelファイルを読み込む
    try:
        df = pd.read_excel(io.BytesIO(content))
    except Exception as e:
        print(f"エラー: {base_filename} の読み込みに失敗しました。理由: {str(e)}")
        continue

    # time列とPeak Strength Time列の処理
    date_columns = ['time', 'Peak Strength Time']
    for col in date_columns:
        if col in df.columns:
            df[col] = df[col].apply(clean_datetime)

    # D列（Nominal stress）を削除
    if 'Nominal stress' in df.columns:
        df = df.drop('Nominal stress', axis=1)

    # ファイル名から拡張子を除去し、新しいCSVファイル名を作成
    csv_filename = os.path.splitext(base_filename)[0] + '.csv'

    # CSVファイルとして保存（UTF-8エンコーディング）
    csv_path = os.path.join(save_dir, csv_filename)
    try:
        # 既存のファイルがある場合は上書き
        df.to_csv(csv_path, index=False, encoding='utf-8', date_format='%Y-%m-%d %H:%M:%S.%f')
        print(f'{base_filename} を {csv_filename} として {save_dir} に保存しました。')
    except Exception as e:
        print(f"エラー: {csv_filename} の保存に失敗しました。理由: {str(e)}")

print("処理が完了しました。")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving B.xlsx to B (1).xlsx
Saving F.xlsx to F (1).xlsx
Saving F-P.xlsx to F-P (1).xlsx
Saving M.xlsx to M (1).xlsx
B.xlsx を B.csv として /content/drive/My Drive/Excel_to_CSV に保存しました。
F.xlsx を F.csv として /content/drive/My Drive/Excel_to_CSV に保存しました。
F-P.xlsx を F-P.csv として /content/drive/My Drive/Excel_to_CSV に保存しました。
M.xlsx を M.csv として /content/drive/My Drive/Excel_to_CSV に保存しました。
処理が完了しました。
